In [23]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json


# Import create_engine
from sqlalchemy import create_engine
from config import key

In [20]:
# Read in WineData.csv
csv_path = "Location_DF.csv"
Location_df = pd.read_csv(csv_path)

In [21]:
Location_df.head()

,Winery,Latitude,Longitude
0,Plantagenet,-34.623193,117.662067
1,Xanadu,-33.982649,115.056427
2,Vasse River,-33.677252,119.188485
3,Robert Oatley,-27.672817,121.628310
4,Picardy,-34.395843,116.128722


In [22]:
skey= "cXNXftKp6WzZUsff1G6pj3wrJ9r1aLTK"

soils_DF = Location_df

soils_DF["soilQualifierName"] = None
soils_DF["waSoilGroupCode"] = None
soils_DF["waSoilGroupName"] = None
soils_DF["landUnitCode"] = None
soils_DF["landUnitName"] = None


for index, row in Location_df.iterrows():

    try:

        target_url = f"https://api.dpird.wa.gov.au/v1/soil/predictions?latitude={row['Latitude']}&longitude={row['Longitude']}"

        try:

            soil_response = requests.get(target_url, headers={"api_key":skey}).json()
            predictions = soil_response["data"]["predictions"][0]
            del predictions["links"]

            # print(predictions)
            soils_DF.loc[index, "soilQualifierName"] = predictions.get("soilQualifierName")
            soils_DF.loc[index, "waSoilGroupCode"] = predictions.get("waSoilGroupCode")
            soils_DF.loc[index, "waSoilGroupName"] = predictions.get("waSoilGroupName")
            soils_DF.loc[index, "landUnitCode"] = predictions.get("landUnitCode")
            soils_DF.loc[index, "landUnitCode"] = predictions.get("landUnitName")
        


        except requests.exceptions.JSONDecodeError as e:
            # print(e)
            soils_DF.loc[index, "soilQualifierName"] = ""
            soils_DF.loc[index, "waSoilGroupCode"] = ""
            soils_DF.loc[index, "waSoilGroupName"] = ""
            soils_DF.loc[index, "landUnitCode"] = ""
            soils_DF.loc[index, "landUnitCode"] = ""




        # break
            

    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
print(soils_DF.head())

          Winery   Latitude   Longitude             soilQualifierName  \
0    Plantagenet -34.623193  117.662067               neutral subsoil   
1         Xanadu -33.982649  115.056427              gravelly subsoil   
2    Vasse River -33.677252  119.188485  differentiation not required   
3  Robert Oatley -27.672817  121.628310                  not assigned   
4        Picardy -34.395843  116.128722               neutral subsoil   

  waSoilGroupCode      waSoilGroupName  landUnitCode landUnitName  
0             302  Duplex sandy gravel   slopes 1-3%         None  
1             103        Semi-wet soil   slopes 1-3%         None  
2             102       Salt lake soil     salt lake         None  
3             522     Red shallow loam  not assigned         None  
4             302  Duplex sandy gravel  slopes 5-10%         None  


In [24]:
soils_DF= soils_DF[["Winery", "Latitude", "Longitude", "soilQualifierName", "waSoilGroupName"]]
soils_DF.head()

,Winery,Latitude,Longitude,soilQualifierName,waSoilGroupName
0,Plantagenet,-34.623193,117.662067,neutral subsoil,Duplex sandy gravel
1,Xanadu,-33.982649,115.056427,gravelly subsoil,Semi-wet soil
2,Vasse River,-33.677252,119.188485,differentiation not required,Salt lake soil
3,Robert Oatley,-27.672817,121.628310,not assigned,Red shallow loam
4,Picardy,-34.395843,116.128722,neutral subsoil,Duplex sandy gravel


In [25]:
clean_soils_df= soils_DF.rename(columns={"Winery":"Winery", "Latitude":"Latitude", "Longitude":"Longitude", "soilQualifierName":"SubSoil_Condition", "waSoilGroupName":"Soil_Type"})
clean_soils_df.head()



,Winery,Latitude,Longitude,SubSoil_Condition,Soil_Type
0,Plantagenet,-34.623193,117.662067,neutral subsoil,Duplex sandy gravel
1,Xanadu,-33.982649,115.056427,gravelly subsoil,Semi-wet soil
2,Vasse River,-33.677252,119.188485,differentiation not required,Salt lake soil
3,Robert Oatley,-27.672817,121.628310,not assigned,Red shallow loam
4,Picardy,-34.395843,116.128722,neutral subsoil,Duplex sandy gravel


In [26]:
clean_soils_df = clean_soils_df[clean_soils_df['Winery'] != "Hope Estate"]
clean_soils_df


,Winery,Latitude,Longitude,SubSoil_Condition,Soil_Type
0,Plantagenet,-34.623193,117.662067,neutral subsoil,Duplex sandy gravel
1,Xanadu,-33.982649,115.056427,gravelly subsoil,Semi-wet soil
2,Vasse River,-33.677252,119.188485,differentiation not required,Salt lake soil
3,Robert Oatley,-27.672817,121.628310,not assigned,Red shallow loam
4,Picardy,-34.395843,116.128722,neutral subsoil,Duplex sandy gravel
5,Frankland Estate,-34.462871,117.013378,neutral subsoil,Duplex sandy gravel
6,West Cape Howe,-34.655733,117.557831,deep sandy duplex,Semi-wet soil
7,Leeuwin Estate,-34.013389,115.065652,neutral subsoil,Duplex sandy gravel
8,Wildberry Estate,-27.672817,121.628310,not assigned,Red shallow loam
9,Vasse Felix,-33.822468,115.037726,neutral subsoil,Loamy gravel


In [27]:
clean_soils_df.to_csv("Soils_DF2.csv", index=False, header=True)

LOADING

In [29]:
user_name = "postgres"
password = key
rds_connection_string = f"{user_name}:{password}@localhost:5432/wa_wine_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [30]:
engine.table_names()

['Winery_Location',
 'Weather_Average_Rainfall',
 'Weather_Average_Solar',
 'Soil_data',
 'Winery_Export_Sales_by_Geolocation',
 'Wine_volume_by_region',
 'Wine_Grape_variety_Sales',
 'Wine_Grape_varierty_Production',
 'Winery_Tasting_Price_data']

In [ ]:
pd.read_sql_query('select * from Soil_data', con=engine).head()